In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import argparse
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import requests
from IPython.core.debugger import set_trace
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
KEY_FILE_LOCATION = '/content/drive/My Drive/Google Analytics API/aq-data-pull-7d38c6520d33.json'
SERVICE_ACCOUNT_EMAIL = 'jack-aq-data@aq-data-pull.iam.gserviceaccount.com'
import warnings
warnings.simplefilter('ignore')

In [0]:
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION, scopes=SCOPES)
    http = credentials.authorize(httplib2.Http())
    # Build the service object.
    analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI,cache_discovery=False)
    return analytics

def print_response(response,date):
    data = response.get('reports', [])[0].get('data', {}).get('rows', [])
    df = pd.DataFrame(data)
    df1 = df.copy()
    try:
        l1 = df1.iloc[0][1]
    except:
        return pd.DataFrame(columns=[date,'dimensions'])
    d1 = l1[0]
    l2 = list(d1.values())
    l3 = l2[0]
    l4 = l3[0]
    df1['metrics'] = df1.iloc[:,1].apply(lambda x: x[0]).apply(lambda x: list(x.values())[0]).apply(lambda x: x[0])
    df1['dimensions'] = df1.iloc[:,0].apply(lambda x: x[0])
    df1 = df1.rename(columns={"metrics": date})
    return df1
def get_report(startDate,endDate,segment,view,pages,analytics=initialize_analyticsreporting(),page_limit=0):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
    if pages==1:
        if page_limit==0:
            print('Please enter pagelimit>0 to use with pages=1')
            return
        else:
            body={
              'reportRequests': [
                  {
                      'viewId': view,
                      'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
                      'metrics': [
                          {'expression': 'ga:users'},
                      ],
                      "dimensions": [
                          {"name": "ga:landingPagePath"},
                          { "name": "ga:segment" }
                      ],
                      "segments": [
                          {'segmentId': segment},
                      ],
                      "pageSize": page_limit,
                      "orderBys": [
                        {
                          "fieldName": "ga:users",
                          "sortOrder": "DESCENDING"
                              }
                      ]
            }]}
    else:
        body={
              'reportRequests': [
                  {
                      'viewId': view,
                      'dateRanges': [{'startDate': startDate, 'endDate': endDate}],
                      'metrics': [
                          {'expression': 'ga:users'},
                      ],
                      "dimensions": [
                          { "name": "ga:segment" }
                      ],
                      "segments": [
                          {'segmentId': segment},
                      ]                              
              
          }]}
    return analytics.reports().batchGet(body=body).execute()

def date_range(start_month,start_year,end_month,end_year):
    
    def month_range(month,year):
        y = str(year)
        if month<10:
            m = '0' + str(month)
        else:
            m = str(month)
        d1 = '01'
        thirty = [9,4,6,11,2]
        if month in thirty:
            if month == 2:
                d_end = '28'
            else:
                d_end = '30'
        else:
            d_end = '31'
        start_string = y+'-'+m+'-'+d1
        end_string = y+'-'+m+'-'+d_end
        return [start_string,end_string]

    m = start_month
    l1 = []
    if start_year == end_year:
        for i in list(range(start_month,end_month+1,1)):
            l1.append(month_range(i,start_year))
        return l1
    for i in list(range(m,13,1)):
        l1.append(month_range(i,start_year))
    
    new_year = start_year+1
    while new_year != end_year:
        for i in list(range(1,13,1)):
            l1.append(month_range(i,new_year))
        new_year+=1
    for i in list(range(1,end_month+1,1)):
        l1.append(month_range(i,new_year))
    return l1                    


def monthly_site_data(dict_of_segments,dates,view): #dictionary with keys=segment name and values=segment definition
    
    def one_dataframe(segment,segment_name,dates,view):   
        master_df = pd.DataFrame()
        for i in dates:
            start = i[0]
            end = i[1]
            df = print_response(get_report(start,end,segment,view,pages=0),start)
            master_df[start] = df[start]
        df1 = master_df.transpose().rename(columns={0:segment_name})
        return df1
    
    if len(dict_of_segments) == 0:
        print('Need at least one arg')
        return
    df1 = pd.DataFrame()
    df_list = []
    if len(dict_of_segments) != 0:
        for ar in dict_of_segments:
            s1 = dict_of_segments[ar]
            s2 = ar
            df_this = one_dataframe(s1,s2,dates,view)
            df1[s2] = df_this[s2]
    return df1

def monthly_page_data(segment,view,page_limit,dates):
    counter = 0
    for months in dates:
        start = months[0]
        end = months[1]        
        df = print_response(get_report(start,end,segment,view,pages=1,page_limit=page_limit),start)
        df.set_index('dimensions',inplace=True)
        if counter == 0:
            master_df = df.copy()
            counter = 1
        else:
            master_df[start] = df[start]
            not_in = []
            for i in list(df.index):
                if i not in list(master_df.index):
                    not_in.append(i)
            master_df = master_df.append(df.loc[not_in],sort=False)
    master_df = master_df.fillna(0).astype(int)
    list_of_index = list(master_df.index)
    no_question_marks = []
    for i in list_of_index:
        if '?' in i:
            no_question_marks.append(i.split('?')[0])
        else:
            no_question_marks.append(i)
    master_df = master_df.rename(index=dict(zip(list_of_index,no_question_marks)))
    master_df.reset_index(inplace=True)
    master_df = master_df.groupby('dimensions',axis='index').sum()
    return master_df

def get_link(body):
    split1 = str(body).split('/')
    if len(split1) == 4:
        domain = str(body).split('/')[-2]
    else:
        domain = 'ibd.netitdontmatter'
    split = str(body).split(domain)
    if len(split)<2:
        return float('NaN')
    else:
        if len(split) == 2:
            return str(domain)+split[1]
        else:
            return 'Data Error: Too Many Splits'
def ga_link(link):
    #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(link, verify=False)
    url = response.url
    part = '/'.join(url.split('/')[3:])
    full = '/'+part
    return full
def get_post_id(link):
    split = str(link).split('=')
    if len(split)<2:
        return float('NaN')
    else:
        if len(split) == 2:
            return split[1]
        else:
            return 'Data Error: Too Many Splits'

In [4]:
!pip install random_user_agent

     |████████████████████████████████| 8.2MB 2.7MB/s 


In [0]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import requests

def ga_link(link):
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100000)
    user_agent = user_agent_rotator.get_random_user_agent()
    headers = {'User-Agent': user_agent}
    response = requests.get(link, verify=False,headers=headers)
    url = response.url
    part = '/'.join(url.split('/')[3:])
    full = '/'+part
    return full

In [0]:
#View IDs
VIEW_ID_PSO = 'ga:122434023'
VIEW_ID_PSA = 'ga:129870027'
VIEW_ID_RA = 'ga:63773775'
VIEW_ID_AS = 'ga:188900143'
VIEW_ID_IBD = 'ga:115327156'
VIEW_ID_MS = 'ga:62646096'
VIEW_ID_T2D = 'ga:71807823'
VIEW_ID_LC = 'ga:136919782'
dict_of_view_ids = {}
dict_of_view_ids['PsA'] = VIEW_ID_PSA
dict_of_view_ids['RA'] = VIEW_ID_RA
dict_of_view_ids['PsO'] = VIEW_ID_PSO
dict_of_view_ids['IBD'] = VIEW_ID_IBD
dict_of_view_ids['AS'] = VIEW_ID_AS
dict_of_view_ids['MS'] = VIEW_ID_MS
dict_of_view_ids['T2D'] = VIEW_ID_T2D
dict_of_view_ids['LC'] = VIEW_ID_LC

#Used for crohns only. Page restrictions and no in app traffic
crohns_cap = 'sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);condition::ga:pagePath=@/caregiver/a-husbands-perspective/,ga:pagePath=@/living/lifestyle-change-saved-life/,ga:pagePath=@/living/intuition/,ga:pagePath=@/living/5-tips-coping-high-school/,ga:pagePath=@/living/flare-ice-hack/,ga:pagePath=@/nutrition/vegan-crohns/,ga:pagePath=@/living/why-not-participate-study/,ga:pagePath=@/living/plan-life-routine/,ga:pagePath=@/nutrition/crohns-and-the-fodmap-elimination-diet/,ga:pagePath=@/living/college-crohns/,ga:pagePath=@/living/elizabeth-letter-to-self/,ga:pagePath=@/living/crohns-disease-vulva/,ga:pagePath=@/living/julie-letter-to-self/,ga:pagePath=@/living/life-hacks-living-with-fistula/,ga:pagePath=@/living/teeth-decay/,ga:pagePath=@/living/jenna-letter-to-self/,ga:pagePath=@/living/paul-letter-to-self/,ga:pagePath=@/living/colonoscopy-much-more-with-ibd/,ga:pagePath=@/living/crohns-disease-women-genitalia/,ga:pagePath=@/living/fear-passing-cd-on/' #Crohns cap performs signifcantly better
crohns_cap_proxy = 'users::sequence::ga:sessions>1;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:country=@United States;condition::ga:pagePath[]/caregiver/a-husbands-perspective/|/living/lifestyle-change-saved-life/|/living/intuition/|/living/5-tips-coping-high-school/|/living/flare-ice-hack/|/nutrition/vegan-crohns/|/living/why-not-participate-study/|/living/plan-life-routine/|/nutrition/crohns-and-the-fodmap-elimination-diet/|/living/college-crohns/;condition::ga:pagePath[]/living/elizabeth-letter-to-self/|/living/crohns-disease-vulva/|/living/julie-letter-to-self/|/living/life-hacks-living-with-fistula/|/living/teeth-decay/|/living/jenna-letter-to-self/|/living/paul-letter-to-self/|/living/colonoscopy-much-more-with-ibd/|/living/crohns-disease-women-genitalia/|/living/fear-passing-cd-on/'
#crohns_cap_proxy = 'users::sequence::ga:sessions>1;ga:sessionDuration>30,ga:pageviews>3;->>ga:pagePath=@/caregiver/a-husbands-perspective/,ga:pagePath=@/living/lifestyle-change-saved-life/,ga:pagePath=@/living/intuition/,ga:pagePath=@/living/5-tips-coping-high-school/,ga:pagePath=@/living/flare-ice-hack/,ga:pagePath=@/nutrition/vegan-crohns/,ga:pagePath=@/living/why-not-participate-study/,ga:pagePath=@/living/plan-life-routine/,ga:pagePath=@/nutrition/crohns-and-the-fodmap-elimination-diet/,ga:pagePath=@/living/college-crohns/,ga:pagePath=@/living/elizabeth-letter-to-self/,ga:pagePath=@/living/crohns-disease-vulva/,ga:pagePath=@/living/julie-letter-to-self/,ga:pagePath=@/living/life-hacks-living-with-fistula/,ga:pagePath=@/living/teeth-decay/,ga:pagePath=@/living/jenna-letter-to-self/,ga:pagePath=@/living/paul-letter-to-self/,ga:pagePath=@/living/colonoscopy-much-more-with-ibd/,ga:pagePath=@/living/crohns-disease-women-genitalia/,ga:pagePath=@/living/fear-passing-cd-on/'
#crohns_cap_proxy = 'users::sequence::ga:sessions>1;ga:pageviews>3,ga:sessionDurationBucket>30;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);condition::ga:pagePath=@/caregiver/a-husbands-perspective/,ga:pagePath=@/living/lifestyle-change-saved-life/,ga:pagePath=@/living/intuition/,ga:pagePath=@/living/5-tips-coping-high-school/,ga:pagePath=@/living/flare-ice-hack/,ga:pagePath=@/nutrition/vegan-crohns/,ga:pagePath=@/living/why-not-participate-study/,ga:pagePath=@/living/plan-life-routine/,ga:pagePath=@/nutrition/crohns-and-the-fodmap-elimination-diet/,ga:pagePath=@/living/college-crohns/,ga:pagePath=@/living/elizabeth-letter-to-self/,ga:pagePath=@/living/crohns-disease-vulva/,ga:pagePath=@/living/julie-letter-to-self/,ga:pagePath=@/living/life-hacks-living-with-fistula/,ga:pagePath=@/living/teeth-decay/,ga:pagePath=@/living/jenna-letter-to-self/,ga:pagePath=@/living/paul-letter-to-self/,ga:pagePath=@/living/colonoscopy-much-more-with-ibd/,ga:pagePath=@/living/crohns-disease-women-genitalia/,ga:pagePath=@/living/fear-passing-cd-on/'
#Restrictions: Everything EXCEPT FB in app traffic
cs_total = 'sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
all_total = 'sessions::condition::ga:country=@United States'

#Restrictions: Only INCLUDE facebook in app traffic. (Used to compare the in app traffic to non-in app traffic)
total_just_fb = 'sessions::condition::ga:country=@United States;condition::ga:browser=@Safari (in-app)'
proxy_just_fb = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:country=@United States;condition::ga:browser=@Safari (in-app)'

#Restrictions: Facebook only. No FB in app traffic. 
total_cs_fb = 'users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy_cs_fb = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy_cs_fb_pvs = 'sessions::sequence::ga:pageviews>1;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'
proxy_cs_fb_ssns = 'users::sequence::ga:sessions>=2;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'

#Restrictions: Promoted FB only. No in app traffic.
promoted_fb = 'users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);sessions::condition::ga:sourceMedium=@facebook.com / promoted'
proxy_promoted = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;users::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app);sessions::condition::ga:sourceMedium=@facebook.com / promoted'

total_fb = 'sessions::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States'
proxy_fb = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States'

bd_facebook_proxy = 'users::sequence::ga:sessions>=2,ga:sessionDuration>120,ga:pageviews>2;sessions::condition::ga:source=@facebook.com;sessions::condition::ga:country=@United States'
bd_site_proxy = 'users::sequence::ga:sessions>=2,ga:sessionDuration>120,ga:pageviews>2;sessions::condition::ga:country=@United States;condition::ga:browser!@Safari (in-app)'


total_organic = 'sessions::condition::ga:channelGrouping=@Organic Search'
proxy_organic = 'users::sequence::ga:sessions>=2;ga:sessionDuration>30,ga:pageviews>3;sessions::condition::ga:channelGrouping=@Organic Search'

In [0]:
dol_crohns = {}
dol_crohns['Proxy FB'] = proxy_cs_fb
dol_crohns['Total FB'] = total_cs_fb
dol_crohns['All FB Users'] = total_fb
dol_crohns['Site Proxy'] = proxy
dol_crohns['Site Total'] = cs_total
dol_crohns['PVS FB'] = proxy_cs_fb_pvs
dol_crohns['SSN FB'] = proxy_cs_fb_ssns

In [0]:
dol_recos = {}
dol_recos['Proxy FB'] = proxy_cs_fb
dol_recos['Total FB'] = total_cs_fb

In [0]:
dol_optimizations = {}
dol_optimizations['Proxy Organic'] = proxy_organic
dol_optimizations['Total Organic'] = total_organic
dol_optimizations['Proxy FB'] = proxy_fb
dol_optimizations['Total FB'] = total_fb

In [0]:
import pickle
class recommendations:
    def __init__(self,condition,rec_month,hist_ids,ids):
        self.condition = condition
        if self.condition=='CD':
            self.view_id = dict_of_view_ids['IBD']
        elif self.condition=='UC':
            self.view_id = dict_of_view_ids['IBD']
        else:
            self.view_id = dict_of_view_ids[self.condition]
        self.rec_month = rec_month
        self.hist_ids = hist_ids
        self.ids = ids
        self.df = None
        self.hist_org_df = None
        self.model_organic = None
        self.hist_org_std = None
        self.org_df = None
        self.ids_ga = None
        self.hist_ids_ga = None
        self.org_hist_ids_ga = None
        self.org_ids_ga = None
    def add_ga_links(self,hist=True):
        hist_ids = self.hist_ids
        ids = self.ids
        if hist==True:
            hist_ids['Link'] = hist_ids['Body'].apply(lambda x: get_link(x))
            hist_ids = hist_ids.loc[~hist_ids['Link'].isnull()]
            hist_ids['GA Link'] = hist_ids['Link'].apply(lambda x: ga_link('https://'+x))
            hist_ids['Post ID'] = hist_ids['Link'].apply(lambda x: get_post_id(x))
            
            hist_ids.to_excel('/Users/jackarmand/Documents/Reccomended Pages/'+self.condition+' GA IDs.xlsx')
            self.hist_ids_ga = hist_ids
        
        ids['Link'] = ids['Body'].apply(lambda x: get_link(x))
        ids = ids.loc[~ids['Link'].isnull()]
        ids['Post ID'] = ids['Link'].apply(lambda x: get_post_id(x))
        ids['GA Link'] = ids['Link'].apply(lambda x: ga_link('https://'+x))
        
        ids.to_excel('/Users/jackarmand/Documents/Reccomended Pages/March Reporting/'+self.condition+' GA March IDs.xlsx')
        self.ids_ga = ids
        
        print('Completed and Saved')
    def add_ga_links_organic(self,org_hist_ids,org_ids,hist=True):
        self.org_hist_ids = org_hist_ids
        self.org_ids = org_ids
        if hist==True:
            hist_ids = self.org_hist_ids.copy()
            hist_ids['Link'] = hist_ids['Post Message'].apply(lambda x: get_link(x))
            hist_ids = hist_ids.loc[~hist_ids['Link'].isnull()]
            hist_ids['GA Link'] = hist_ids['Link'].apply(lambda x: ga_link('https://'+x))
            self.org_hist_ids_ga = hist_ids
            hist_ids.to_excel('/Users/jackarmand/Documents/Reccomended Pages/'+self.condition+' GA Organic IDs.xlsx')            
        
        ids = self.org_ids.copy()
        ids['Link'] = ids['Post Message'].apply(lambda x: get_link(x))
        ids = ids.loc[~ids['Link'].isnull()]
        ids['GA Link'] = ids['Link'].apply(lambda x: ga_link('https://'+x))
        
        self.org_ids_ga = ids
        ids.to_excel('/Users/jackarmand/Documents/Reccomended Pages/March Reporting/'+self.condition+' GA Organic March IDs.xlsx')
        
    def info_on_urls(self,urls,dates):        
        hist_df = monthly_page_data(proxy_fb,self.view_id,500,dates)
        hist_df2 = monthly_page_data(total_fb,self.view_id,500,dates)
        hist_df2.columns = [x+'-total' for x in hist_df2.columns]

        total = pd.concat([hist_df2,hist_df],axis=1)
        dates = hist_df.columns
        for i in dates:
            total[i+'-percent'] = total[i]/total[i+'-total']
        total = total.fillna(0)
        tcols = [x for x in total.columns if 'total' in x]
        d = {}
        list_of_urls = []
        for i in list(total.index):
            for x in dates:
                if total.loc[i][x+'-total']>100:
                    d[(i,x)] = ([total.loc[i][x+'-total']][0],total.loc[i][x+'-percent'])

        df = pd.DataFrame(d).transpose()
        df.columns=['Total','Percent']
        df = df.sort_values('Total')
        
        
        X = np.log(np.array(df['Total']).reshape(-1,1))
        pred = self.model.predict(X)
        df['Pred'] = pred
        df['Diff'] = df['Percent']-df['Pred']
        df = df.sort_values('Diff',ascending=False)
        
        df = df.reset_index(level=1)
        df = df.loc[urls]
        
        self.urls_df = df
        
    def initialize_recs(self,dates):
        self.initialize_dates = dates
        
        hist_df = monthly_page_data(proxy_fb,self.view_id,500,self.initialize_dates)
        hist_df2 = monthly_page_data(total_fb,self.view_id,500,self.initialize_dates)
        hist_df2.columns = [x+'-total' for x in hist_df2.columns]

        total = pd.concat([hist_df2,hist_df],axis=1)
        dates = hist_df.columns
        for i in dates:
            total[i+'-percent'] = total[i]/total[i+'-total']
        total = total.fillna(0)
        tcols = [x for x in total.columns if 'total' in x]
        d = {}
        list_of_urls = []
        for i in list(total.index):
            for x in dates:
                if total.loc[i][x+'-total']>100:
                    d[(i,x)] = ([total.loc[i][x+'-total']][0],total.loc[i][x+'-percent'])

        df = pd.DataFrame(d).transpose()
        df.columns=['Total','Percent']
        df = df.sort_values('Total')
        
        X = np.log(np.array(df['Total']).reshape(-1,1))
        y = list(df['Percent'])
        ols = LinearRegression()
        model = ols.fit(X,y)
        pred = model.predict(X)
        df['Pred'] = pred
        df['Diff'] = df['Percent']-df['Pred']
        df = df.sort_values('Diff',ascending=False)
        
        hist_ids = self.hist_ids_ga
        hist_ids1 = hist_ids.groupby('GA Link').first()
        hist_ids1 = hist_ids1.loc[~hist_ids1['Post ID'].isnull()]
        
        df = df.reset_index(level=1)
        df['Ad Name'] = hist_ids1['Ad Name']
        df['Post ID'] = hist_ids1['Post ID']
        
        
        self.hist_df = df
        self.model = model
        self.hist_std = df['Diff'].std()
    def make_recs(self,proxy_segment,total_segment):
        df = monthly_page_data(proxy_segment,self.view_id,500,self.rec_month)
        df2 = monthly_page_data(total_segment,self.view_id,500,self.rec_month)
        df2.columns=[x+'-total' for x in df2.columns]
        
        total = pd.concat([df2,df],axis=1)
        dates = df.columns
        for i in dates:
            total[i+'-percent'] = total[i]/total[i+'-total']
        total = total.fillna(0)
        tcols = [x for x in total.columns if 'total' in x]
        d = {}
        list_of_urls = []
        for i in list(total.index):
            for x in dates:
                if total.loc[i][x+'-total']>500:
                    d[(i,x)] = ([total.loc[i][x+'-total']][0],total.loc[i][x+'-percent'])

        df = pd.DataFrame(d).transpose()
        df.columns=['Total','Percent']
        df = df.replace([np.inf,-np.inf],np.nan)
        df = df.dropna()
        df = df.sort_values('Total')
        X = np.array(df['Total']).reshape(-1,1)
        df['Pred'] = self.model.predict(X)
        df['Diff'] = df['Percent']-df['Pred']
        df = df.sort_values('Diff',ascending=False)
        
        
        
        ids = self.ids_ga
        ids1 = ids.groupby('GA Link').first()
        #ids1 = ids1.loc[~ids1['Post ID'].isnull()]
        
        fb = df.loc[df.reset_index(level=1,drop=False).index.isin(list(ids1.index))]
        fb = fb.dropna()
        fb = fb.reset_index(level=1)
        fb['Ad Name'] = ids1['Ad Name']
        fb['Post ID'] = ids1['Post ID']
        fb['Preview Link'] = ids1['Preview Link']
        fb['Pred'] = self.model.predict(np.log(np.array(fb['Total']).reshape(-1,1)))
        fb['Diff'] = fb['Percent']-fb['Pred']
        fb['Score'] = round((fb['Diff']/self.hist_std)*2+5,1)
        
        self.df = fb.sort_values('Score',ascending=False)  
    def initialize_organic_recs(self,organic_init_dates):
        self.organic_init_dates = organic_init_dates
        hist_df = monthly_page_data(proxy_fb,self.view_id,1000,self.organic_init_dates)
        hist_df2 = monthly_page_data(total_fb,self.view_id,1000,self.organic_init_dates)
        hist_df2.columns = [x+'-total' for x in hist_df2.columns]

        total = pd.concat([hist_df2,hist_df],axis=1)
        dates = hist_df.columns
        for i in dates:
            total[i+'-percent'] = total[i]/total[i+'-total']
        total = total.fillna(0)
        tcols = [x for x in total.columns if 'total' in x]
        d = {}
        list_of_urls = []
        for i in list(total.index):
            for x in dates:
                if total.loc[i][x+'-total']>20:
                    d[(i,x)] = ([total.loc[i][x+'-total']][0],total.loc[i][x+'-percent'])

        df = pd.DataFrame(d).transpose()
        df.columns = ['Total','Percent']
        df = df.sort_values('Total')
        
        X = np.log(np.array(df['Total']).reshape(-1,1))
        y = list(df['Percent'])
        ols = LinearRegression()
        model = ols.fit(X,y)
        pred = model.predict(X)
        df['Pred'] = pred
        df['Diff'] = df['Percent']-df['Pred']
        df = df.sort_values('Diff',ascending=False)
        
        hist_ids = self.org_hist_ids_ga.copy()
        hist_ids1 = hist_ids.groupby('GA Link').first()
        hist_ids1['Post ID'] = hist_ids1['Link'].apply(lambda x: get_post_id(x))
        
        df = df.loc[df.reset_index(level=1,drop=False).index.isin(list(hist_ids['GA Link']))]
        df = df.loc[~df.reset_index(level=1,drop=False).index.isin(list(self.hist_df.index))]
        df = df.reset_index(level=1)
        df['Preview Link'] = hist_ids1['Permalink']
        df['Post ID'] = hist_ids1['Post ID']
        df['Score'] = round((df['Diff']/df['Diff'].std())*2+5,1)
        
        self.hist_org_df = df
        self.model_organic = model
        self.hist_org_std = df['Diff'].std()
        
    def make_organic_recs(self,organic_rec_dates):
        self.organic_rec_dates = organic_rec_dates
        hist_df = monthly_page_data(proxy_fb,self.view_id,2000,self.organic_rec_dates)
        hist_df2 = monthly_page_data(total_fb,self.view_id,2000,self.organic_rec_dates)
        hist_df2.columns = [x+'-total' for x in hist_df2.columns]

        total = pd.concat([hist_df2,hist_df],axis=1)
        dates = hist_df.columns
        for i in dates:
            total[i+'-percent'] = total[i]/total[i+'-total']
        total = total.fillna(0)
        tcols = [x for x in total.columns if 'total' in x]
        d = {}
        list_of_urls = []
        for i in list(total.index):
            for x in dates:
                if total.loc[i][x+'-total']>20:
                    d[(i,x)] = ([total.loc[i][x+'-total']][0],total.loc[i][x+'-percent'])

        df = pd.DataFrame(d).transpose()
        df.columns = ['Total','Percent']
        df = df.sort_values('Total')
        
        X = np.log(np.array(df['Total']).reshape(-1,1))
        pred = self.model_organic.predict(X)
        df['Pred'] = pred
        df['Diff'] = df['Percent']-df['Pred']
        df = df.sort_values('Diff',ascending=False)
        
        ids = self.org_ids_ga.copy()
        ids1 = ids.groupby('GA Link').first()
        ids1['Post ID'] = ids1['Link'].apply(lambda x: get_post_id(x))
        
        df = df.loc[df.reset_index(level=1,drop=False).index.isin(list(ids['GA Link']))]
        df = df.loc[~df.reset_index(level=1,drop=False).index.isin(list(self.hist_df.index))]
        df = df.loc[~df.reset_index(level=1,drop=False).index.isin(list(self.df.index))]
        
        df = df.reset_index(level=1)
        df['Preview Link'] = ids1['Permalink']
        df['Post ID'] = ids1['Post ID']
        df['Score'] = round((df['Diff']/self.hist_org_std)*2+5,1)
        
        self.org_df = df
    def finalize_df(self):
        org = self.org_df.copy()
        org = org.reset_index()
        org = org.groupby('index').first().sort_values('Score',ascending=False)
        org = org[['Score','Post ID','Preview Link']]
        org = org.loc[org['Score']>7]
        self.organic_output = org
        
        prom = self.df.copy()
        prom = prom[['Ad Name','Post ID','Preview Link','Score']]
        self.promoted_output = prom
        
        prom.to_excel('/Users/jackarmand/Documents/Reccomended Pages/March Reporting/Final Copies/'+self.condition+' Promoted.xlsx')
        org.to_excel('/Users/jackarmand/Documents/Reccomended Pages/March Reporting/Final Copies/'+self.condition+' Organic.xlsx')       
        

In [0]:
root_dir = '/content/drive/My Drive/Google Analytics API/Reccomended Pages'
mar = 'March Reporting/'
def rec_wrapper(condition,rec_month,hist=True,hist_org=True):
    
    recs = recommendations(condition,date_range(rec_month,2020,rec_month,2020),
                          pd.read_excel(root_dir+condition+' IDs.xlsx'),
                           pd.read_excel(root_dir+mar+condition+' March IDs.xlsx'))
    if hist==False:
        recs.hist_ids_ga = pd.read_excel(root_dir+condition+' GA IDs.xlsx')
        recs.ids_ga = pd.read_excel(root_dir+mar+condition+' GA March IDs.xlsx')
    if hist_org==False:
        recs.org_hist_ids_ga = pd.read_excel(root_dir+condition+' GA Organic IDs.xlsx')
        recs.org_ids_ga = pd.read_excel(root_dir+mar+condition+' GA Organic March IDs.xlsx')
    if hist==True:
        recs.add_ga_links(hist=True)
        recs.hist_ids_ga = pd.read_excel(root_dir+condition+' GA IDs.xlsx')
    if hist_org==True:
        recs.add_ga_links_organic(pd.read_excel(root_dir+condition+' Organic IDs.xlsx'),
                                 pd.read_excel(root_dir+mar+condition+' Organic March IDs.xlsx'),hist=True)
        recs.org_hist_ids_ga = pd.read_excel(root_dir+condition+' GA Organic IDs.xlsx')
    recs.initialize_recs(date_range(7,2019,12,2019))
    recs.make_recs(proxy_fb,total_fb)
    recs.initialize_organic_recs(date_range(9,2019,12,2019))
    recs.make_organic_recs(date_range(rec_month,2020,rec_month,2020))
    recs.finalize_df()
    print(condition,'Completed')

In [0]:
conditions = ['PsA','PsO','CD','UC','RA','AS']
month = 3
for condition in conditions:
  rec_wrapper(condition,month)

Completed and Saved
UC Completed
